In [1]:
import boto3
import zipfile
import os

# Especificar o perfil do AWS CLI
session = boto3.Session(profile_name='CRISTIAN_AWS')

# Criar cliente do Lambda com a sessão específica
lambda_client = session.client('lambda')

# Função para criar um arquivo zip contendo o código da Lambda
def criar_arquivo_zip():
    # Código da função Lambda
    codigo_lambda = """
import json
import boto3
import uuid
from datetime import datetime

# Conectar ao DynamoDB
dynamodb = boto3.resource('dynamodb')
tabela = dynamodb.Table('FeedbackSimulado')

def lambda_handler(event, context):
    # Obter os dados do corpo da requisição (via API Gateway)
    body = json.loads(event['body'])

    pergunta_id = body.get('pergunta_id')
    comentario = body.get('comentario')
    satisfacao = body.get('satisfacao')

    # Gerar um ID único para o feedback
    feedback_id = str(uuid.uuid4())
    
    # Registrar a data e hora
    data_hora = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

    # Dados a serem inseridos no DynamoDB
    item = {
        'FeedbackID': feedback_id,
        'PerguntaID': pergunta_id,
        'Comentario': comentario,
        'Satisfacao': satisfacao,
        'DataHora': data_hora
    }

    # Inserir no DynamoDB
    try:
        tabela.put_item(Item=item)
        return {
            'statusCode': 200,
            'body': json.dumps({'message': 'Feedback enviado com sucesso!'})
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }
    """

    # Criar um arquivo ZIP contendo o código da Lambda
    with open("lambda_function.py", "w") as arquivo_lambda:
        arquivo_lambda.write(codigo_lambda)
    
    # Compactar o arquivo
    with zipfile.ZipFile("lambda_function.zip", "w") as zipf:
        zipf.write("lambda_function.py")

# Função para criar a Lambda na AWS
def criar_lambda():
    criar_arquivo_zip()
    
    # Ler o arquivo ZIP criado
    with open("lambda_function.zip", 'rb') as f:
        codigo_zip = f.read()

    try:
        response = lambda_client.create_function(
            FunctionName='FeedbackSimuladoFunction',
            Runtime='python3.9',
            Role='arn:aws:iam::058264285974:role/LambdaDynamoDBRole',  # Substitua pelo ARN correto da Role
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': codigo_zip},
            Description='Função Lambda para inserir feedback no DynamoDB',
            Timeout=15,
            MemorySize=128,
            Publish=True
        )
        print("Função Lambda criada com sucesso!")
        print(response)
    except Exception as e:
        print(f"Erro ao criar a função Lambda: {e}")

# Chamar a função para criar a Lambda
criar_lambda()


Função Lambda criada com sucesso!
{'ResponseMetadata': {'RequestId': '69e65146-284d-4389-ae9a-2c0c7d96a007', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 15 Oct 2024 03:48:49 GMT', 'content-type': 'application/json', 'content-length': '1405', 'connection': 'keep-alive', 'x-amzn-requestid': '69e65146-284d-4389-ae9a-2c0c7d96a007'}, 'RetryAttempts': 0}, 'FunctionName': 'FeedbackSimuladoFunction', 'FunctionArn': 'arn:aws:lambda:us-east-1:058264285974:function:FeedbackSimuladoFunction', 'Runtime': 'python3.9', 'Role': 'arn:aws:iam::058264285974:role/LambdaDynamoDBRole', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 1363, 'Description': 'Função Lambda para inserir feedback no DynamoDB', 'Timeout': 15, 'MemorySize': 128, 'LastModified': '2024-10-15T03:48:48.869+0000', 'CodeSha256': 'q3nHbevsN1EfiAoYeYlZAfueNnP4rN00aJfaqVYc4L0=', 'Version': '1', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '8c2cab34-88c2-46ee-b9da-66bd02d147c7', 'State': 'Pending', 'StateReason': 